In [1]:
from mixtral_v1 import MixtralDecoderLayerTimed


def modify_mixtral_moe_block(type__: str):
    if type__ == 'decoder':
        import transformers.models.mixtral.modeling_mixtral as mix_models
        mix_models.MixtralDecoderLayer = MixtralDecoderLayerTimed
    else:
        pass
modify_mixtral_moe_block('decoder')

/home/lkx/data/qwen_moe/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
model_id = 'mistralai/Mixtral-8x7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, dtype=torch.bfloat16, device_map='auto', max_memory={
        0: '64GB'
    })

Loading checkpoint shards: 100%|██████████| 19/19 [00:09<00:00,  1.94it/s]
Some parameters are on the meta device because they were offloaded to the disk.


In [3]:
print(model.hf_device_map)
model.eval()

{'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 0, 'model.layers.11': 0, 'model.layers.12': 0, 'model.layers.13': 0, 'model.layers.14': 0, 'model.layers.15': 0, 'model.layers.16': 0, 'model.layers.17': 0, 'model.layers.18': 0, 'model.layers.19': 0, 'model.layers.20': 0, 'model.layers.21.self_attn': 0, 'model.layers.21.block_sparse_moe.gate': 0, 'model.layers.21.block_sparse_moe.experts.0': 0, 'model.layers.21.block_sparse_moe.experts.1': 0, 'model.layers.21.block_sparse_moe.experts.2': 0, 'model.layers.21.block_sparse_moe.experts.3': 0, 'model.layers.21.block_sparse_moe.experts.4': 0, 'model.layers.21.block_sparse_moe.experts.5': 0, 'model.layers.21.block_sparse_moe.experts.6.w1': 0, 'model.layers.21.block_sparse_moe.experts.6.w2': 0, 'model.layers.21.block_sparse_moe.experts.6.w3':

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayerTimed(
        (self_attn): MixtralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBlockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=14336, bias=False)
              (w2): Linear(in_features=14336, out_features=4096, bias=False)
              (w3): Linear(in_features=4096, out_features=14336, bias=False)
              (act_fn): SiLUActivation()
           

In [4]:
prompt = "what is the difference between mixtral and mistral?"
model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

In [5]:
from torch.profiler import profile, ProfilerActivity
with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    record_shapes=False, with_stack=True, profile_memory=False
) as prof:
    with torch.no_grad():
        # _ = model(next_token, past_key_values=past_kv, use_cache=True)
        _ = model.generate(
            **model_inputs,
            max_new_tokens=10,
            do_sample=True,
        )
prof.export_chrome_trace("trace/trace_mixtral_64GB.json")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
generate_ids = model.generate(
    **model_inputs,
    max_new_tokens=10,
    do_sample=True,
)
generate_ids

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[    1,   767,   349,   272,  5133,  1444,  6750,   434,   282,   304,
          5710,  1650, 28804,    13,    13, 23809, 28725,    13,    13,  1014,
         11398,   464, 23350, 28733, 28742,  2825,  1545,   843,  2508,   304,
           791, 28719,  3162, 28723,  1263,  2757,  3475, 28719,  3162, 28733,
           715, 28809,   349,   264, 20521,   302,   544,   272,  1722, 28725,
           304,   272, 11398,  3475, 28719,   392, 28733, 28809,   349,  1307,
           354,   521,  6206,   442, 16502,  2330,  1722, 28723,    13,    13,
          2324,   541,   938,  1167,   989, 11398,   274,   297,  1581, 11846,
         28723,    13, 26388,  2825,   464,   532,  6750,   647,  1312,  5710,
          2825,   464, 28728, 20346,  4135,    13, 26388, 28733,   434,   282,
         28746,  1791,   347,  9430, 28723,    13, 28755,   392, 28733,  1650,
           327,   330,   363, 20346,  5535, 28723,    13,    13, 28737,  3317,
           378,  2870,   378,  3081, 28723,    13,  

In [5]:
model.hf_device_map

{'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 0,
 'model.layers.4': 0,
 'model.layers.5': 0,
 'model.layers.6': 0,
 'model.layers.7': 0,
 'model.layers.8': 0,
 'model.layers.9': 0,
 'model.layers.10': 0,
 'model.layers.11': 0,
 'model.layers.12': 0,
 'model.layers.13': 0,
 'model.layers.14': 0,
 'model.layers.15': 0,
 'model.layers.16': 0,
 'model.layers.17': 0,
 'model.layers.18': 0,
 'model.layers.19': 0,
 'model.layers.20': 'disk',
 'model.layers.21': 'disk',
 'model.layers.22': 'disk',
 'model.layers.23': 'disk',
 'model.layers.24': 'disk',
 'model.layers.25': 'disk',
 'model.layers.26': 'disk',
 'model.layers.27': 'disk',
 'model.layers.28': 'disk',
 'model.layers.29': 'disk',
 'model.layers.30': 'disk',
 'model.layers.31': 'disk',
 'model.norm': 'disk',
 'model.rotary_emb': 'disk',
 'lm_head': 'disk'}

In [6]:
output_ids = generate_ids[0][model_inputs['input_ids'].shape[1]:]
output_text = tokenizer.decode(output_ids, skip_special_tokens=True)
output_text

'\n- They both mean west wind. Mixtral is a mix of the directions mistral and tramontane, the former coming from the north (northwesterly) and the latter from the north east (northeasterly). Neither of the two winds mentioned are local winds (and thus not included in a glossary of local winds); the mistake comes from a website which misquotes Wikipedia, where it says the mixtral is the direction northwesterly and southeasterly (mistral + tramontane).\n\nAs mistral is often considered to be west to northwest, the same word often means westwardly.\n\nSource: Wikipedia.\n\nThis answer is:'